In [3]:
import os
import cv2
import random
import numpy as np
from pathlib import Path
from sklearn.utils import shuffle

tanhDerivative = lambda x : 1 - np.tanh(x)**2

"""
Function to get feature vectors and numeric labels (-1 or 1) of the images in the given path
Only images belonging to the two given classes will be returned
Params:
    folderPath (str): Path of the subset (train/test)
    class1 (str): Folder name of the class to be numerically labeled as -1
    class2 (str): Folder name of the class to be numerically labeled as 1
Returns:
    inputs (list), t (list): Lists of feature vectors and numeric labels
"""
def readDataSet(folderPath, class1, class2):
    inputs = []
    t = []
    for root, dirs, files in os.walk(folderPath):
        for file in files:
            if file.endswith('.jpg'):
                filePath = os.path.join(root, file)
                pathParts = Path(root).parts
                className = pathParts[-1]
                if className != class1 and className != class2:
                    continue
                img = cv2.imread(filePath)
                imgResized = cv2.resize(img, dsize=(64,64), interpolation=cv2.INTER_CUBIC)
                imgVector = imgResized.flatten()
                inputs.append(imgVector)
                if className == class1:
                    t.append(-1) # Class1 is labeled as -1 because the tanh activation function gives output in range [-1,1]
                elif className == class2:
                    t.append(1)
    return inputs, t

"""
Function to train perceptron with tanh activation function
Params:
    inputs (list): Feature vectors of train samples
    t (list): Numeric labels of train samples
    weights (numpy.array): Weights with initial random small values
    rho (float): Learning rate
    iterNo (int): Number of iterations
Returns:
    weights (numpy.array): Weights after the training
"""
def trainPerceptron(inputs, t, weights, rho, iterNo):
    gradient = np.zeros(len(weights)) # May be removed, new values will be assigned to the gradient in the loop
    
    for iteration in range(iterNo):
        for index, x in enumerate(inputs):
            summation = np.matmul(np.transpose(weights), x) # Feed forward: Calculate the sum
            y = np.tanh(summation) # Feed forward: Activation function
            error = t[index] - y # Feed backward: Loss computation
            gradient = rho * error * tanhDerivative(y) * x # Feed backward: Calculate the gradient
            weights = weights + gradient # Feed backward: Update the weights
    return weights

"""
Function to test perceptron with tanh activation function
Params:
    sampleTest (list): Feature vectors of test samples
    weights (numpy.array): Trained weights
Returns:
    predictions (list): Predicted labels
"""
def testPerceptron(sampleTest, weights):
    predictions = []
    
    for x in sampleTest:
        summation = np.matmul(np.transpose(weights), x) # Feed forward: Calculate the sum
        y = np.tanh(summation) # Feed forward: Activation function
        predictions.append(y)
    return predictions
            
# Train and test paths
trainPath = r'/content/drive/MyDrive/YuksekLisans/DLAPP/HW3/CaltechTinySplit/train'
valPath = r'/content/drive/MyDrive/YuksekLisans/DLAPP/HW3/CaltechTinySplit/val'
testPath = r'/content/drive/MyDrive/YuksekLisans/DLAPP/HW3/CaltechTinySplit/test'

# Feature vectors and labels for train and test
trainX, trainY = readDataSet(trainPath, 'cannon', 'cellphone')
valX, valY = readDataSet(valPath, 'cannon', 'cellphone')
testX, testY = readDataSet(testPath, 'cannon', 'cellphone')

# Bias term
for i in range(len(trainX)):
    trainX[i] = np.append(trainX[i], 1)

# Bias term
for i in range(len(valX)):
    valX[i] = np.append(valX[i], 1)

# Bias term    
for i in range(len(testX)):
    testX[i] = np.append(testX[i], 1)

# Initialized weights with a random small value
weights = np.zeros(len(trainX[0]))
for i in range(len(weights)):
    weights[i] = random.randint(1,9) * 10**-2

trainX, trainY = shuffle(trainX, trainY) # Shuffle the train data
weights = trainPerceptron(trainX, trainY, weights, 0.001, 1000) # Train
np.save('weights.npy', weights) # Saving weights to file


In [4]:
weights = np.load('weights.npy') # Loading weights from file

valPredictions = testPerceptron(valX, weights) # Predicting the validation set

# Compare the actual validation labels with the predictions
print('Actual\tPred.\tComparison')
for i in range(len(valPredictions)):
    print('{}\t{}\t{}'.format(valY[i], valPredictions[i], valY[i]==valPredictions[i]))

Actual	Pred.	Comparison
-1	-1.0	True
-1	-1.0	True
-1	-1.0	True
-1	-1.0	True
1	1.0	True
1	1.0	True
1	1.0	True
1	1.0	True
1	1.0	True


In [5]:
weights = np.load('weights.npy') # Loading weights from file

testPredictions = testPerceptron(testX, weights) # Predicting the test set

# Compare the actual test labels with the predictions
print('Actual\tPred.\tComparison')
for i in range(len(testPredictions)):
    print('{}\t{}\t{}'.format(testY[i], testPredictions[i], testY[i]==testPredictions[i]))

Actual	Pred.	Comparison
1	1.0	True
1	1.0	True
1	1.0	True
1	1.0	True
1	1.0	True
1	1.0	True
1	1.0	True
-1	-1.0	True
-1	-1.0	True
-1	-1.0	True
-1	-1.0	True
-1	-1.0	True
